# Module 5 Unit 2
## Improving neural networks with regularisation

### Regression problem


In [ ]:
# Import libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np

## Highway accidents


In [ ]:
# Import data
df = pd.read_csv('Highway.txt', delimiter = " ")

In [ ]:
# Explore the size of the data set
df.shape

In [ ]:
# Explore the type of data and feature names
df.sample(10, random_state=0)

In [ ]:
# Split data into features (X) and responses (y)
X = df.iloc[:, 1:11] 
y = df.loc[:,"lgRate"]

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
# Change the array shape of the output from a dataframe single column vector
# to a contiguous flattened array
y = np.ravel(y)

In [ ]:
# Split the data into the training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [ ]:
# Scale the data
scaler = StandardScaler()  

# Remember to fit using only the training data
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  

# Apply the same transformation to the test data
X_test = scaler.transform(X_test)

In [ ]:
# Fit and check MSE before regularisation
reg = MLPRegressor(max_iter=3000, hidden_layer_sizes=(5,5), random_state=1) 
reg.fit(X_train, y_train)

# Predict
y_pred = reg.predict(X_test)

# MSE before regularisation
mean_squared_error(y_pred, y_test)

The regularisation parameter is used to "apply the brakes". A sequence of numbers that are exponentially decaying is created to account for the complexity near 0. A larger value represents a simpler problem complexity. 

In [ ]:
# Find the optimum regularisation parameter
reg_par = [np.e**n for n in np.arange(-3,5,0.5)]

# Optimise the neural network regularisation
validation_scores = {}
print(" alpha  |  Sq.Error")   
for param in reg_par:
    reg = MLPRegressor(max_iter=6000, hidden_layer_sizes=(5,5), 
                       alpha=param, random_state=1)
    score = cross_val_score(estimator=reg, X=X_train, y=y_train, 
                            cv=3, scoring="neg_mean_squared_error")
    validation_scores[param] = -score.mean()
    print("%0.5f |  %0.6f" % (param, -score.mean()))   

# Plot the error function    
plt.plot([np.log(i) for i in validation_scores.keys()], list(validation_scores.values()))
plt.xlabel("Ln of alpha")
plt.ylabel("Mean Sq Error")


In [ ]:
# Find the regularisation parameter with the lowest error
print("The lowest cross validation error is %f" % min(validation_scores.values()))        
print("This corresponds to regularisation parameter e**%s" % 
      ([np.log(name) for name, score in validation_scores.items()
                         if score==min(validation_scores.values())][0]))

The regularisation parameter found via cross-validation is $e^{1.5}$. This means that the task is relatively simple.

In [ ]:
# Fit the data with the best parameter
reg = MLPRegressor(max_iter=6000, hidden_layer_sizes=(5,5), 
                   alpha=np.e**(1.5), random_state=1)
reg.fit(X_train, y_train)
# Does not fully converge without changing max_iter

In [ ]:
# Predict
y_pred = reg.predict(X_test)

# MSE final
mean_squared_error(y_pred, y_test)

Notice the decrease in error before and after regularisation.

For production purposes, you would then use a regressor fitted on the entire data set (X,y), not just the test data.

In [ ]:
# Visualise by drawing a response function
# Observe each of the predictors individually vs lgRate

for variable in X.columns:
    # Copy the dataframe, so as not to change the original, and obtain medians
    X_design = X.copy()
    X_design_vec = pd.DataFrame(X_design.median()).transpose()

    # Grab the min and max of desired variable and set up a sequence
    min_res = min(X.loc[:, variable])
    max_res = max(X.loc[:, variable])
    seq = np.linspace(start=min_res, stop=max_res, num=50)

    # Set up a list of moving resultants
    to_predict = []
    for result in seq:
        X_design_vec.loc[0, variable] = result
        to_predict.append(X_design_vec.copy())

    # Convert back to DataFrame
    to_predict = pd.concat(to_predict)

    # Scale and predict
    to_predict = scaler.transform(to_predict)
    predictions = reg.predict(to_predict)

    # Plot 
    plt.plot(seq, predictions)
    plt.xlabel(variable)
    plt.ylabel("lgRate")
    plt.title("lgRate vs. " + variable)
    plt.show()

## Practice

Find the optimum regularisation parameter if `hidden_layer_sizes = (8,3)`.

In [ ]:
# Find optimum regularisation parameter
# Hint: Change the range in the following line of code.
# reg_par = [np.e**n for n in np.arange(?,?,?)]

# YOUR CODE HERE


# Optimise neural network regularisation
validation_scores = {}
print(" alpha  |  Sq.Error")   
for param in reg_par:
    reg = MLPRegressor(max_iter=6000, hidden_layer_sizes=(8,3), 
                       alpha=param, random_state=1)
    score = cross_val_score(estimator=reg, X=X_train, y=y_train, 
                            cv=3, scoring="neg_mean_squared_error")
    validation_scores[param] = -score.mean()
    print("%0.5f |  %0.6f" % (param, -score.mean())) 
    
# Plot the error function    
plt.plot([np.log(i) for i in validation_scores.keys()], list(validation_scores.values()))
plt.xlabel("Ln of alpha")
plt.ylabel("Mean Sq Error")

# Find the regularisation parameter with the lowest error
print("The lowest cross validation error is %f" % min(validation_scores.values()))        
print("This corresponds to regularisation parameter e**%s" % 
      ([np.log(name) for name, score in validation_scores.items()
                         if score==min(validation_scores.values())][0]))